<a href="https://colab.research.google.com/github/chaerui7967/stock_predict_news_and_youtube/blob/master/%ED%98%84%EB%8C%80%EC%B0%A8_%ED%86%B5%ED%95%A9%ED%85%8C%EC%9D%B4%EB%B8%94_10%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘 시작

## 라이브러리 설치

In [1]:
!pip install finance-datareader
!pip install pykrx
!pip install pymysql
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

     |████████████████████████████████| 70 kB 3.7 MB/s 
     |████████████████████████████████| 60 kB 6.8 MB/s 
     |████████████████████████████████| 251 kB 30.7 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 19.4 MB 122 kB/s 
     |████████████████████████████████| 448 kB 32.6 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


## 패키지 & 함수 로딩

In [2]:
# 필요 라이브러리
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 주식가격 로딩
import FinanceDataReader as fdr
from pykrx import stock

import datetime

In [3]:
def money_sur(stock_name, start, end):

    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')

    return data

In [4]:
def reverse_min_max_scaling(org_x, x): #종가 예측값
    org_x_np = np.asarray(org_x) 
    x_np = np.asarray(x)
    return (x_np * (org_x_np.max() - org_x_np.min())) + org_x_np.min()

## 통합테이블 생성

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# total_table = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/total_table.csv') # 기존 테이블

In [7]:
# 구성
dic = {'code' : [],
       'date' : [],
       'lstm' : [],
       'arima' : [],
       'fbprophet' : [],
       'RL' : [],
    #    'mail_news' : [],
    #    'mail_news_nsi' : [],
    #    'asia_news' : [],
    #    'asia_news_nsi': [],
    #    'youtube_sam' : [],
    #    'youtube_su' : [],
    #    'youtube_han' : [],
       'close' : []}

total_table = pd.DataFrame(dic)

In [8]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'close'], dtype='object')

In [9]:
total_table

,code,date,lstm,arima,fbprophet,RL,close


## LSTM 예측결과

### 사용 데이터 로드 및 전처리

In [10]:
start_date = (datetime.datetime.now() - datetime.timedelta(40)).strftime("%Y%m%d")
end_date = datetime.datetime.now().strftime("%Y%m%d")
lstm = money_sur('005380', start_date, end_date)

In [11]:
lstm = lstm.set_index('Date')

In [12]:
lstm.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-27,209000,209500,205000,208500,502218,-0.007143,250888,38.23,0.83,5454,1.44,3000
2021-08-30,211000,212500,210000,210000,409963,0.007194,250888,38.50,0.84,5454,1.43,3000
2021-08-31,210000,212500,208000,212500,777103,0.011905,250888,38.96,0.85,5454,1.41,3000
2021-09-01,214000,218500,213500,215000,855937,0.011765,250888,39.42,0.86,5454,1.40,3000
2021-09-02,214000,215000,210000,211500,656283,-0.016279,250888,38.78,0.84,5454,1.42,3000


In [13]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
lstm_sc = sc.fit_transform(lstm)

In [14]:
scale_cols = lstm.columns
lstm_sc_df = pd.DataFrame(lstm_sc, columns=[scale_cols], index=lstm.index)

In [15]:
lstm_sc_df.head()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS
Date,,,,,,,,,,,,
2021-08-27,0.765957,0.617021,0.653061,0.697674,0.154990,0.487523,0.0,0.704715,0.666666,0.0,0.266667,0.0
2021-08-30,0.851064,0.744681,0.857143,0.767442,0.040383,0.763172,0.0,0.771712,0.777777,0.0,0.200000,0.0
2021-08-31,0.808511,0.744681,0.775510,0.883721,0.496477,0.853738,0.0,0.885856,0.888889,0.0,0.066667,0.0
2021-09-01,0.978723,1.000000,1.000000,1.000000,0.594411,0.851045,0.0,1.000000,1.000000,0.0,0.000000,0.0
2021-09-02,0.978723,0.851064,0.857143,0.837209,0.346384,0.311868,0.0,0.841191,0.777777,0.0,0.133333,0.0


In [16]:
for i in scale_cols:
  for s in range(1, 11):  
    lstm_sc_df[(i+'shift_{}'.format(s),)] = lstm_sc_df[(i,)].shift(s)

In [17]:
lstm_sc_df.dropna()

,Open,High,Low,Close,Volume,Change,BPS,PER,PBR,EPS,DIV,DPS,Openshift_1,Openshift_2,Openshift_3,Openshift_4,Openshift_5,Openshift_6,Openshift_7,Openshift_8,Openshift_9,Openshift_10,Highshift_1,Highshift_2,Highshift_3,Highshift_4,Highshift_5,Highshift_6,Highshift_7,Highshift_8,Highshift_9,Highshift_10,Lowshift_1,Lowshift_2,Lowshift_3,Lowshift_4,Lowshift_5,Lowshift_6,Lowshift_7,Lowshift_8,...,PBRshift_1,PBRshift_2,PBRshift_3,PBRshift_4,PBRshift_5,PBRshift_6,PBRshift_7,PBRshift_8,PBRshift_9,PBRshift_10,EPSshift_1,EPSshift_2,EPSshift_3,EPSshift_4,EPSshift_5,EPSshift_6,EPSshift_7,EPSshift_8,EPSshift_9,EPSshift_10,DIVshift_1,DIVshift_2,DIVshift_3,DIVshift_4,DIVshift_5,DIVshift_6,DIVshift_7,DIVshift_8,DIVshift_9,DIVshift_10,DPSshift_1,DPSshift_2,DPSshift_3,DPSshift_4,DPSshift_5,DPSshift_6,DPSshift_7,DPSshift_8,DPSshift_9,DPSshift_10
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-09-10,0.744681,0.617021,0.612245,0.604651,0.373745,0.440429,0.0,0.612903,0.555555,0.0,0.333334,0.0,0.914894,0.829787,1.000000,1.000000,0.936170,0.978723,0.978723,0.808511,0.851064,0.765957,0.744681,0.829787,0.829787,0.893617,0.872340,0.851064,1.000000,0.744681,0.744681,0.617021,0.775510,0.857143,0.918367,1.000000,0.938776,0.857143,1.000000,0.775510,...,0.666666,0.888889,0.888889,1.000000,0.888889,0.777777,1.000000,0.888889,0.777777,0.666666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.266667,0.066667,0.066667,0.000000,0.000000,0.133333,0.000000,0.066667,0.200000,0.266667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-13,0.659574,0.489362,0.612245,0.534884,0.264798,0.485195,0.0,0.545906,0.555555,0.0,0.400000,0.0,0.744681,0.914894,0.829787,1.000000,1.000000,0.936170,0.978723,0.978723,0.808511,0.851064,0.617021,0.744681,0.829787,0.829787,0.893617,0.872340,0.851064,1.000000,0.744681,0.744681,0.612245,0.775510,0.857143,0.918367,1.000000,0.938776,0.857143,1.000000,...,0.555555,0.666666,0.888889,0.888889,1.000000,0.888889,0.777777,1.000000,0.888889,0.777777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333334,0.266667,0.066667,0.066667,0.000000,0.000000,0.133333,0.000000,0.066667,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-14,0.659574,0.702128,0.693878,0.720930,0.426880,1.000000,0.0,0.727047,0.666666,0.0,0.266667,0.0,0.659574,0.744681,0.914894,0.829787,1.000000,1.000000,0.936170,0.978723,0.978723,0.808511,0.489362,0.617021,0.744681,0.829787,0.829787,0.893617,0.872340,0.851064,1.000000,0.744681,0.612245,0.612245,0.775510,0.857143,0.918367,1.000000,0.938776,0.857143,...,0.555555,0.555555,0.666666,0.888889,0.888889,1.000000,0.888889,0.777777,1.000000,0.888889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400000,0.333334,0.266667,0.066667,0.066667,0.000000,0.000000,0.133333,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-15,0.723404,0.702128,0.775510,0.744186,0.118513,0.670849,0.0,0.749380,0.777777,0.0,0.200000,0.0,0.659574,0.659574,0.744681,0.914894,0.829787,1.000000,1.000000,0.936170,0.978723,0.978723,0.702128,0.489362,0.617021,0.744681,0.829787,0.829787,0.893617,0.872340,0.851064,1.000000,0.693878,0.612245,0.612245,0.775510,0.857143,0.918367,1.000000,0.938776,...,0.666666,0.555555,0.555555,0.666666,0.888889,0.888889,1.000000,0.888889,0.777777,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.266667,0.400000,0.333334,0.266667,0.066667,0.066667,0.000000,0.000000,0.133333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-16,0.808511,0.680851,0.714286,0.674419,0.048224,0.487195,0.0,0.682382,0.666666,0.0,0.266667,0.0,0.723404,0.659574,0.659574,0.744681,0.914894,0.829787,1.000000,1.000000,0.936170,0.978723,0.702128,0.702128,0.489362,0.617021,0.744681,0.829787,0.829787,0.893617,0.872340,0.851064,0.775510,0.693878,0.612245,0.612245,0.775510,0.857143,0.918367,1.000000,...,0.777777,0.666666,0.555555,0.555555,0.666666,0.888889,0.888889,1.000000,0.888889,0.777777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.266667,0.400000,0.333334,0.266667,0.066667,0.066667,0.000000,0.000000,0.133333,0.0,0.0,0.0,0.0,0

In [18]:
X_test = lstm_sc_df.dropna().drop(columns=scale_cols, axis=1)
y_test = lstm_sc_df.dropna()[['Close']]

X_test= X_test.values
y_test = y_test.values

In [19]:
X_test

array([[0.91489362, 0.82978723, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.74468085, 0.91489362, 0.82978723, ..., 0.        , 0.        ,
        0.        ],
       [0.65957447, 0.74468085, 0.91489362, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.38297872, 0.27659574, 0.55319149, ..., 0.        , 0.        ,
        0.        ],
       [0.29787234, 0.38297872, 0.27659574, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.29787234, 0.38297872, ..., 0.        , 0.        ,
        0.        ]])

In [20]:
X_test_t = X_test.reshape(X_test.shape[0], 10, 12)

### 모델 로드

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda, Flatten, Dropout
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(LSTM(64,
               input_shape = (10, 12),
               return_sequences= True))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'tanh'))
model.add(LSTM(64,
               return_sequences = False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1))


lstm_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/LSTM/'
hun_lstm = 'lstm_hundai.h5'

model.load_weights(lstm_path+hun_lstm)

### 모델 결과

In [22]:
pred = model.predict(X_test_t)

In [23]:
pred.shape, y_test.shape

((15, 1), (15, 1))

In [24]:
pred_act = reverse_min_max_scaling(lstm['Close'], pred)

In [25]:
total_table['date'] = lstm.index[-10:]
total_table['lstm'] = pred_act[-10:]
total_table['close'] = lstm.Close[-10:].values.reshape(-1,1)
total_table['code'] = '005380'
total_table['change'] = lstm.Change[-10:].values.reshape(-1,1)

In [26]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005380,2021-09-17,208241.223633,NaN,NaN,NaN,209000,0.004808
1,005380,2021-09-23,207684.588867,NaN,NaN,NaN,208500,-0.002392
2,005380,2021-09-24,207527.737305,NaN,NaN,NaN,207000,-0.007194
3,005380,2021-09-27,207766.279297,NaN,NaN,NaN,204000,-0.014493
4,005380,2021-09-28,206633.387695,NaN,NaN,NaN,201500,-0.012255
5,005380,2021-09-29,205588.459961,NaN,NaN,NaN,201500,0.000000
6,005380,2021-09-30,204709.598633,NaN,NaN,NaN,200000,-0.007444
7,005380,2021-10-01,203983.612305,NaN,NaN,NaN,193500,-0.032500
8,005380,2021-10-05,202261.120117,NaN,NaN,NaN,193500,0.000000
9,005380,2021-10-06,200252.648438,NaN,NaN,NaN,194500,0.005168


## arima 예측결과

In [27]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('005380', '2018', end_date)

In [28]:
df = samsung_df[['Date','Close']]
df.columns = ['Date', 'Price']

In [29]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
# 학습, 테스트 데이터 분리
for ru in range(-10,0):
    rus = ru+1
    train = df[:ru]
    test = df[ru:]

    model = ARIMA(train.Price.values, order = (2,1,2))
    model_fit = model.fit(trend = 'nc', full_output = True, disp = True)

    fore = model_fit.forecast(steps=1) # 오늘 예측

    # 오늘 예측 데이터
    pred_arima_y = fore[0].tolist()

    # 실제 오늘 데이터 
    test_y = test.Price.values

    # # 신뢰구간
    # # 예측 데이터 최소값
    # pred_y_lower = []
    # # 예측 데이터 최대값
    # pred_y_upper = []

    # for lower_upper in fore[2]:
    #     lower = lower_upper[0]
    #     upper = lower_upper[1]
    #     pred_y_lower.append(lower)
    #     pred_y_upper.append(upper)
    if rus == 0:
        total_table['arima'].iloc[ru:] = pred_arima_y
    else:
        total_table['arima'].iloc[ru:rus] = pred_arima_y

print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                  924
Model:                 ARIMA(2, 1, 2)   Log Likelihood               -8858.557
Method:                       css-mle   S.D. of innovations           3526.889
Date:                Wed, 06 Oct 2021   AIC                          17727.114
Time:                        02:08:24   BIC                          17751.257
Sample:                             1   HQIC                         17736.325
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1.D.y     -0.8447      0.075    -11.261      0.000      -0.992      -0.698
ar.L2.D.y     -0.7712      0.089     -8.636      0.000      -0.946      -0.596
ma.L1.D.y      0.9465      0.068     13.985      0.0

In [30]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005380,2021-09-17,208241.223633,208203.587355,NaN,NaN,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,NaN,NaN,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,NaN,NaN,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,NaN,NaN,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,NaN,NaN,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,NaN,NaN,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,NaN,NaN,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,NaN,NaN,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,NaN,NaN,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,NaN,NaN,194500,0.005168


## fbprophet 예측

In [31]:
end_date = datetime.datetime.now().strftime("%Y%m%d")
samsung_df = money_sur('005380', start = '20180101', end = end_date)
df1 = samsung_df[['Date','Close']]
df1.columns = ['ds', 'y']

from fbprophet import Prophet

prophet = Prophet(seasonality_mode = 'multiplicative',
                 yearly_seasonality=True, 
                 weekly_seasonality=True,
                 daily_seasonality=True,
                 changepoint_prior_scale=0.6)

prophet.fit(df1)

# 10일단위로 예측값을 가져옴
future_data = prophet.make_future_dataframe(periods = 10, freq = 'd')
forecast_data = prophet.predict(future_data)

In [32]:
list(total_table.date.values)

[numpy.datetime64('2021-09-17T00:00:00.000000000'),
 numpy.datetime64('2021-09-23T00:00:00.000000000'),
 numpy.datetime64('2021-09-24T00:00:00.000000000'),
 numpy.datetime64('2021-09-27T00:00:00.000000000'),
 numpy.datetime64('2021-09-28T00:00:00.000000000'),
 numpy.datetime64('2021-09-29T00:00:00.000000000'),
 numpy.datetime64('2021-09-30T00:00:00.000000000'),
 numpy.datetime64('2021-10-01T00:00:00.000000000'),
 numpy.datetime64('2021-10-05T00:00:00.000000000'),
 numpy.datetime64('2021-10-06T00:00:00.000000000')]

In [33]:
total_table['fbprophet'] = forecast_data[forecast_data['ds'].isin(list(total_table.date.values))]['yhat'].values

In [34]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,NaN,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,NaN,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,NaN,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,NaN,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,NaN,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,NaN,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,NaN,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,NaN,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,NaN,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,NaN,194500,0.005168


In [35]:
# total_table.to_csv('./total_table.csv', index =False)

## RL 결과

### 데이터 로딩 및 함수 로딩

In [36]:
import time
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

# 금융정보 조회하기
def dataset_loaderKR(stock_name, start, end, train_ratio=0.8):
    start_date = start
    end_date = end
    sample_code = stock_name

    dataset = fdr.DataReader(sample_code, start = start_date, end = end_date )
    dataset = dataset.reset_index()
    df = stock.get_market_fundamental_by_date(fromdate=start_date, todate=end_date, ticker=sample_code)
    df = df.reset_index()
    df.columns = ['Date','BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']
    data = pd.merge(dataset, df, on='Date')
    date_split = data.index[int(train_ratio*len(data))] 

    return data[:date_split], data[date_split:], date_split


# ENV 설정
class Environment1:
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs

    def step(self, act):
        reward = 0
        
        # action
        # 0: Idle
        # 1: 매수
        # 2: 매도
        if act == 1: #매수
            self.positions.append(self.data.iloc[self.t, :]['Close'])
        elif act == 2: # 매도
            if len(self.positions) == 0:
                reward = -1
            else:
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Close'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['Close'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['Close'] - self.data.iloc[(self.t-1), :]['Close'])
        if (self.t==len(self.data)-1):
            self.done=True
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        #print ("t={%d}, done={%str}"%(self.t,self.done))
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

# MODEL
class Q_Network(nn.Module):
    def __init__(self,obs_len,hidden_size,actions_n):
        super(Q_Network,self).__init__()
        self.fc_val = nn.Sequential(
            nn.Linear(obs_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, actions_n)
        )
    def forward(self,x):
        h = self.fc_val(x)
        return (h)

In [37]:
(train, test, date_split) = dataset_loaderKR('005380', start = '20180101', end = end_date)

### 모델 로딩 및 결과

In [38]:
rl_path = '/content/drive/My Drive/Final PJT - 업빛투/Regression/RL/현대차_결과/'
hun_rl = 'hyundai_rl_재무정보.h5'

In [39]:
# 모델 로딩
model = torch.load(rl_path+hun_rl)

test_env = Environment1(test)
pobs = test_env.reset()
test_acts = []
test_rewards = []

for _ in range(len(test_env.data)-1):
    pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
    pact = np.argmax(pact.data)
    test_acts.append(pact.item())
            
    obs, reward, done = test_env.step(pact.numpy())
    test_rewards.append(reward)

    pobs = obs

# sell remaining stocks
profits = 0
for p in test_env.positions:
	profits += (test_env.data.iloc[test_env.t, :]['Close'] - p)

test_env.profits += profits
test_env.positions = []

test_profits = test_env.profits
pact = model(torch.from_numpy(np.array(pobs, dtype=np.float32).reshape(1, -1)))
pact = np.argmax(pact.data)
next_action = pact.item()
test['Action'] = test_acts + [next_action]


print( )
print( test )
print( )
print("결과")
print( "손익결과 $%.2f" % round(test_profits,2) )
# 0: Idle
# 1: 매수
# 2: 매도


          Date    Open    High     Low   Close  ...   PBR    EPS   DIV   DPS  Action
740 2021-01-06  209500  211500  202500  203000  ...  0.80  11310  1.97  4000       1
741 2021-01-07  206000  210500  204500  206000  ...  0.81  11310  1.94  4000       1
742 2021-01-08  240500  257000  225000  246000  ...  0.97  11310  1.63  4000       0
743 2021-01-11  256500  289000  256000  267500  ...  1.06  11310  1.50  4000       0
744 2021-01-12  263500  274500  250500  261000  ...  1.03  11310  1.53  4000       0
..         ...     ...     ...     ...     ...  ...   ...    ...   ...   ...     ...
921 2021-09-29  197500  202000  197500  201500  ...  0.80   5454  1.49  3000       2
922 2021-09-30  200000  201000  199000  200000  ...  0.80   5454  1.50  3000       1
923 2021-10-01  198000  199000  192000  193500  ...  0.77   5454  1.55  3000       2
924 2021-10-05  191000  195000  189000  193500  ...  0.77   5454  1.55  3000       1
925 2021-10-06  195000  197500  192500  194500  ...  0.77   5454

In [40]:
test['Action'] = np.where(test['Action'] == 1,1,
                          np.where(test['Action'] == 2,-1,0))

In [41]:
test['Action'][-10:]

916   -1
917   -1
918    1
919   -1
920    1
921   -1
922    1
923   -1
924    1
925    1
Name: Action, dtype: int64

In [42]:
total_table['RL'] = test[test['Date'].isin(list(total_table.date.values))]['Action'].values

In [43]:
total_table

,code,date,lstm,arima,fbprophet,RL,close,change
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,1,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,1,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,1,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,1,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,1,194500,0.005168


## News 긍부정 결과
- '매일경제', '아시아경제'

In [44]:
# corp_list = ['samsung', 'hyundai', 'lg', 'sk', 'celltrion'] 참고용
stock_num_list = ['005930', '005380', '051910', '000660', '068270']
news = ['maeil_news_craw', 'asia_news_craw']
start = (datetime.datetime.now() - datetime.timedelta(20)).strftime("%Y-%m-%d").replace('-','')
end = datetime.datetime.now().strftime("%Y-%m-%d").replace('-', '')

In [45]:
# news_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/Final Data/1. samsung_score.csv') # 연습용

### 관련 함수 및 패키지 로드

In [46]:
import pymysql
import sys
import re

In [47]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

In [48]:
def news_db(news, stock):
    db = pymysql.connect(
            user='root', 
            passwd='1234', 
            host='3.35.70.166', 
            db='proj', 
            charset = 'utf8'
        )

    cursor = db.cursor(pymysql.cursors.DictCursor)
    news1 = news + '_news_craw'

    sql = "select * from {0}_{1} where (length(date)=10) and (date between {2}00 and {3}23)".format(news1, stock, start, end)
    cursor.execute(sql)
    result = cursor.fetchall()

    # DataFrame으로 변경
    df = pd.DataFrame(result)

    db.close()

    df.rename(columns={'date': 'datetime'}, inplace=True)

    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df['date'] = df['datetime'].str[0:4] + '-' + df['datetime'].str[4:6] + '-' + df['datetime'].str[6:8]
    df['date'] = pd.to_datetime(df['date'])

    # 결측치 제거 → 데이터 로드 시 완료했기 때문에 그다지 필요하지 않은 과정
    df = df.dropna()

    # 시간순으로 정렬
    df.sort_values('datetime', inplace=True)
    df.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)

    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])

    # Stop Words List에 각 매체명 추가
    except_media_list = ['매일경제', '매일', '경제', 'maeil', 'MK', 'mk',
                         '아시아경제', '아시아', 'Asia', 'ASIA', 'asia',
                         '삼프로TV', '삼프로', 'TV',
                         '슈카월드', '슈카', '월드'
                         '한국경제TV', '한국']
    for word in except_media_list:
        stopwords.append(word)
    
    #### 2. Preprocessing
    '''감성 어휘 사전 : negative / positive
       뉴스 데이터 : samsung / hyundai / lg / sk
       주식 데이터 : stock_samsung / stock_hyundai / stock_lg / stock_sk
       공휴일 데이터 : holidays'''
    
    
    ### 1) 뉴스 데이터 날짜 조정
    
    ## 1-1)업로드 시각 컬럼 추가
    df['time'] = df['datetime'].str[-2:]
    
    ## 1-2) 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']

    for j in range(len(df['time'])):
        if df['time'][j] in after_market:
            df['date'][j] += datetime.timedelta(1)
        else:
            pass
    
    ## 1-3) 텍스트 전처리
    # \n, \t, \r 제거
    df['text'] = df['text'].str.replace('[\n|\t|\r]', '')
    df['text'] = df['text'].str.replace('\[|\]', '')
    df['text'] = df['text'].str.replace('\<.*\>', '')
    df['text'] = df['text'].str.replace('\\', '')
    df['text'] = df['text'].str.replace('이미지', '')


    ### 2) 주말 및 공휴일 제외
    
    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df[df['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df[df['date'].isin(market_closed_list)]['date'].index:
            df['date'][hoil] += datetime.timedelta(1)
    
    ### 3) Tokenization 컬럼 추가
    
    df['Tokenization'] = 0
    rows = df.shape[0]
    for row in range(rows):
        hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                                    # 정규 표현식 → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
        result = hangeul.sub('', df['text'][row])   # 위에 설정한 hangeul 규칙을 text에 적용
        okt = Okt()                                                               # 형태소 추출
        nouns = okt.nouns(df['text'][row])
        nouns = [x for x in nouns if len(x) > 1]                                  # 한 글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]                          # 불용어 제거
        
        corpus = " ".join(nouns)                                                  # List를 String으로 변환
        df['Tokenization'][row] = corpus


    # 점수 산출

    df['Positive_Score'] = 0
    df['Negative_Score'] = 0
    df['Ratio'] = 0.1
    df['Pred'] = 0
    df['NSI'] = 0.1
    
    for score in range(len(df)):
        pos_score = 0 ; neg_score = 0
        
        for token in range(len(df['Tokenization'][score].split())):
            if df['Tokenization'][score].split()[token] in positive:
                pos_score += 1
            elif df['Tokenization'][score].split()[token] in negative:
                neg_score += 1
            else:
                pass
        
        df['Positive_Score'][score] = pos_score
        df['Negative_Score'][score] = neg_score
        
        # 긍정과 부정의 비율
        if (pos_score==0) and (neg_score==0):
            df['Ratio'][score] = 0.5   # 둘 다 0일 경우에는 긍정으로 가정
        else:
            df['Ratio'][score] = pos_score / (pos_score + neg_score)
        
        # 예측 결과
        if df['Ratio'][score]>=0.5:
            df['Pred'][score] = 1
        else:
            df['Pred'][score] = -1
        
        # 뉴스심리지수(NSI) 계산
        if (pos_score==0) and (neg_score==0):
            df['NSI'][score] = 101
        else:
            df['NSI'][score] = (pos_score - neg_score) / (pos_score + neg_score) * 100 + 100
    
    return df

### 매일경제

In [49]:
# mail_df = news_df[(news_df['news'] == '매일경제') & (news_df['st_cd'] == 5930)]

In [50]:
mail_df = news_db('maeil', '005380')

In [51]:
mail_df.head()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
0,현대차,005380,매일경제,2021091600,[사설] 첫차 내놓은 광주모터스 무노조 성공모델 기대한다,http://news.mk.co.kr/newsRead.php?no=893411&ye...,광주형 일자리 모델인 광주글로벌모터스(GGM)가 15일...,2021-09-16,00,광주 일자리 모델 광주 글로벌 모터 스포츠 유틸리티 캐스퍼 시작 양산 광주 지역 정...,2,2,0.5,1,100.0
1,현대차,005380,매일경제,2021091609,"현대차 아이오닉 5, 주한 인도네시아 대사관 관용차로 달린다",http://news.mk.co.kr/newsRead.php?no=894124&ye...,"(왼쪽부터) 현대차 김경수 상무, 현대차 김창범 자문역, 우마르 하디(Umar Ha...",2021-09-16,09,왼쪽 현대차 김경수 상무 현대차 김창범 자문 우마르 하디 주한 인도네시아 대사 젤다...,0,0,0.5,1,101.0
2,현대차,005380,매일경제,2021091609,재외 인니대사관 관용차로 첫 전기차…뭘까,http://news.mk.co.kr/newsRead.php?no=894102&ye...,현대자동차가 지난 15일 서울 영등포구 주한 인도네시아 대사관에 전기차 `아이오닉5...,2021-09-16,09,자동차 지난 서울 영등포구 주한 인도네시아 대사관 전기차 아이오 전달 왼쪽 김경수 ...,1,0,1.0,1,200.0
3,현대차,005380,매일경제,2021091612,현대차 오너만의 통합 고객 서비스 앱 '마이현대 2.0' 출시,http://news.mk.co.kr/newsRead.php?no=895248&ye...,현대자동차가 자사 차량 오너들의 필수 고객 서비스 앱 ‘마이현대(myHyundai)...,2021-09-16,12,자동차 자사 차량 필수 고객 서비스 마이 현대 실용 마이 현대 버전 마이 현대 멤버...,1,0,1.0,1,200.0
4,현대차,005380,매일경제,2021091616,현대차·기아 유럽서 첫 10%대 점유율,http://news.mk.co.kr/newsRead.php?no=896760&ye...,현대차그룹이 유럽시장에서 월간 기준으로 처음 두자릿수 시장 점유율을 기록했다.16일...,2021-09-17,16,현대차 그룹 유럽 시장 월간 기준 처음 자릿수 시장 점유 기록 유럽 자동차 공업 협...,0,0,0.5,1,101.0


In [52]:
mail_ratio = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
mail_nsi = mail_df[mail_df['date'].isin(list(total_table.date.values))][['date','NSI']]

In [53]:
mail_ratio = mail_ratio.groupby(['date']).mean().reset_index()
mail_nsi = mail_nsi.groupby(['date']).mean().reset_index()

In [54]:
mail_ratio.columns = ['date', 'mail_news']
mail_nsi.columns = ['date', 'mail_news_nsi']

In [55]:
total_table = pd.merge(total_table, mail_ratio, on='date', how='left')
total_table = pd.merge(total_table, mail_nsi, on='date', how='left')

In [56]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'close', 'change']]

In [57]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,close,change
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,0.633333,127.000000,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,1,0.429420,85.948523,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,194500,0.005168


### 아시아

In [58]:
# asia_df = news_df[(news_df['news'] == '아시아경제') & (news_df['st_cd'] == 5930)]

In [59]:
asia_df = news_db('asia', '005380')

In [60]:
asia_df.tail()

,st_n,st_cd,news,datetime,title,url,text,date,time,Tokenization,Positive_Score,Negative_Score,Ratio,Pred,NSI
106,현대차,005380,아시아경제,2021100510,코스피 3000선 붕괴…먹는 코로나19 치료제에 바이오주 급락,https://view.asiae.co.kr/article/2021100510301...,"코스닥도 하락 출발…3%대까지 낙폭 커져 코스피가 3,000선 아래로 떨어진 5일 ...",2021-10-05,10,코스닥 하락 출발 낙폭 코스피 아래 서울 중구 하나은행 딜링룸 모습 코스피 지수 전...,0,11,0.000000,-1,0.000000
107,현대차,005380,아시아경제,2021100511,美中에서 연일 터지는 악재에…3000선 아래로 추락한 코스피,https://view.asiae.co.kr/article/2021100511383...,美 바이든 행정부의 인프라 법안 표결 10월말로 연기홍콩증시에서 中 부동산 대형업체...,2021-10-05,11,바이든 행정부 인프라 법안 표결 홍콩 증시 부동산 대형 업체 헝다 그룹 거래 잠정 ...,0,9,0.000000,-1,0.000000
108,현대차,005380,아시아경제,2021100513,"[특징주]디스플레이텍, 현대차그룹-한충전 전기차 충전인프라 본격화",https://view.asiae.co.kr/article/2021100513392...,"아시아경제 박형수 기자 디스플레이텍디스플레이텍066670코스닥증권정보현재가8,880...",2021-10-05,13,박형수 기자 디스 레이텍 디스 레이텍 코스닥 증권 정보 현재 전일 대비 거래 전일 ...,5,0,1.000000,1,200.000000
109,현대차,005380,아시아경제,2021100513,오후 들어 진정하는 국내 증시…여전히 3000선 하회하는 코스피,https://view.asiae.co.kr/article/2021100514004...,"미국 제약회사 머크, 먹는 코로나19 치료제 개발…의약품 업종지수 6% 하락 코스피...",2021-10-05,13,미국 제약회사 머크 코로나 치료 개발 의약품 업종 지수 하락 코스피 아래 서울 중구...,1,8,0.111111,-1,22.222222
110,현대차,005380,아시아경제,2021100517,美中에 흔들리는 국내 증시…코스피 3000선 아래서 장 마감,https://view.asiae.co.kr/article/2021100517080...,국내 제약 관련 종목들 단체로 부진…의약품 업종 지수 7.99% 하락 출처=연합뉴스...,2021-10-06,17,국내 제약 관련 종목 단체 부진 의약품 업종 지수 하락 출처 연합뉴스 몃낫 븘씠 공...,1,11,0.083333,-1,16.666667


In [61]:
asia_ratio = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','Ratio']]
asia_nsi = asia_df[asia_df['date'].isin(list(total_table.date.values))][['date','NSI']]

asia_ratio = asia_ratio.groupby(['date']).mean().reset_index()
asia_nsi = asia_nsi.groupby(['date']).mean().reset_index()

asia_ratio.columns = ['date', 'asia_news']
asia_nsi.columns = ['date', 'asia_news_nsi']

In [62]:
total_table = pd.merge(total_table, asia_ratio, on='date', how='left')
total_table = pd.merge(total_table, asia_nsi, on='date', how='left')

In [63]:
# 컬럼 리셋
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news', 'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close', 'change']]

In [64]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close,change
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,0.643590,128.884615,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,0.581148,116.324881,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,0.413591,82.718254,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,0.395833,79.416667,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,0.534722,106.944444,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,0.633333,127.000000,0.439311,88.262285,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,0.722222,144.777778,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,0.596032,119.206349,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,1,0.429420,85.948523,0.412575,82.765031,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,0.083333,16.666667,194500,0.005168


In [65]:
# total_table.to_csv('total_table.csv', index=False)

## youtube 긍부정 결과
- ['한국경제TV', '슈카월드', '삼프로TV_경제의신과함께']

In [66]:
# # 이전 라벨링 데이터
# youtube_df = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/data/youtube_naver_labeling.csv')

In [67]:
# youtube_df.ch_nm.unique()

### youtube 데이터 전체 로드

In [68]:
def youtube_db():

    com_list = {'하이닉스' : '000660',
            '현대차' : '005380',
            '삼성전자' : '005930',
            'LG화학' : '051910',
            '셀트리온' : '068270'}
    media_list = {'youtube_한국경제_':'hk_',
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}
    df_list = []
    for n,m in media_list.items():
        for i,j in com_list.items():

            db = pymysql.connect(
                user='root', 
                passwd='1234', 
                host='3.35.70.166', 
                db='proj', 
                charset = 'utf8'
            )

            cursor = db.cursor(pymysql.cursors.DictCursor)
            sql = "select * from youtube_"+ m + j +" where length(date) = 10"
            
            cursor.execute(sql)

            result = cursor.fetchall()

            # 데이터 프레임으로 변경
            globals() ['df_'+ m + j ] = pd.DataFrame(result)
            print('df_'+ m + j)
            df_list.append(globals() ['df_'+ m + j ])
            
            db.close()
    
    # print(df_list)
    df_total = pd.concat(df_list)
    
    df_total.rename(columns={'date': 'datetime'}, inplace=True)
    # DataFrame 형태를 통일하기 위해 date 컬럼 추가
    df_total['date'] = df_total['datetime'].str[0:4] + '-' + df_total['datetime'].str[4:6] + '-' + df_total['datetime'].str[6:8]
    df_total['date'] = pd.to_datetime(df_total['date'])

    # 시간순으로 정렬
    df_total.reset_index(inplace=True, drop=True)

    ### 4) Holidays
    db = pymysql.connect(user='root',
                         passwd='1234', 
                         host='3.35.70.166', 
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    # 4-1) 주말 및 공휴일 데이터
    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()

    holidays = pd.DataFrame(result)
    db.close()

    holidays['date'] = pd.to_datetime(holidays['date'])

    ## 2-1) 주말 및 공휴일만 추출
    market_closed =holidays[holidays['holiday']=="O"].reset_index(drop=True)
    
    ## 2-3) 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    ## 2-4) iteration limit 조정
    limit_number = 15000
    sys.setrecursionlimit(limit_number)
    
    
    while len(df_total[df_total['date'].isin(market_closed_list)]['date']) != 0:
        for hoil in df_total[df_total['date'].isin(market_closed_list)]['date'].index:
            df_total['date'][hoil] += datetime.timedelta(1)

    # 간단한 전처리
    df_total['text'] = df_total['text'].str.replace('\[|\]','')
    df_total['text'] = df_total['text'].str.replace('[a-z]','')
    df_total['text'] = df_total['text'].str.replace('[A-Z]','')
    df_total['text'] = df_total['text'].str.replace('으','')
    df_total['text'] = df_total['text'].str.replace('음악','')
    df_total['text'] = df_total['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df_total['text'] = df_total['text'].str.replace('^ +', "")

    return df_total

In [69]:
df_total = youtube_db()

df_hk_000660
df_hk_005380
df_hk_005930
df_hk_051910
df_hk_068270
df_sampro_000660
df_sampro_005380
df_sampro_005930
df_sampro_051910
df_sampro_068270
df_suka_000660
df_suka_005380
df_suka_005930
df_suka_051910
df_suka_068270


### 라벨링 패키지 및 모델 로딩

In [70]:
# 라벨 훈련 데이터
X_train = []
aa = pd.read_csv('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/train_label.csv').drop('Unnamed: 0',1)
aa.x = aa.x.str.replace('\[|\]','')
aa.x = aa.x.str.replace("'","")
aa.x = aa.x.str.replace(" ","")

for i in aa.x:
    temp = i.split(',')
    X_train.append(temp)

In [71]:
len(X_train)

145393

In [72]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
tokenizer = Tokenizer(19196)
tokenizer.fit_on_texts(X_train)
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [73]:
max_len = 200

In [74]:
# loaded_model = load_model('./best_model.h5')

In [75]:
loaded_model = load_model('/content/drive/My Drive/Final PJT - 업빛투/감성분석/youtube/best_model.h5')

In [76]:
def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측

    if (score > 0.5):
        # print("{:.2f}% 확률로 긍정 리뷰\n".format(score * 100))
        return score, 1
    elif (score == 0.5):
        return score, 0
    else:
        # print("{:.2f}% 확률로 부정 리뷰\n".format((1 - score) * 100))
        return score, -1

In [77]:
# sentiment_predict('꿀잼') 연습

### 삼프로

In [78]:
sampro_df = df_total[(df_total.ch_nm == '삼프로TV_경제의신과함께') & (df_total.st_cd == '005380')]

In [79]:
sampro_df = sampro_df[sampro_df.date.isin(total_table.date)]

In [80]:
s_text = sampro_df.text.values

In [81]:
scores = [] # 긍정 확률
for i in s_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [82]:
len(sampro_df),len(scores)

(16, 16)

In [83]:
sampro_df['sampro_youtube'] = np.array(scores).reshape(-1,1)
sampro_score = sampro_df.groupby(['date']).mean().reset_index()[['date','sampro_youtube']]

In [84]:
total_table = pd.merge(total_table, sampro_score, on='date',how='left')

In [85]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,close,change,sampro_youtube
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,0.643590,128.884615,209000,0.004808,NaN
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,0.581148,116.324881,208500,-0.002392,0.513158
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,0.413591,82.718254,207000,-0.007194,0.043642
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,0.395833,79.416667,204000,-0.014493,0.560342
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,0.534722,106.944444,201500,-0.012255,0.410304
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,0.633333,127.000000,0.439311,88.262285,201500,0.000000,0.284152
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,0.722222,144.777778,200000,-0.007444,0.828288
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,0.596032,119.206349,193500,-0.032500,0.092859
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,1,0.429420,85.948523,0.412575,82.765031,193500,0.000000,0.141176
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,0.083333,16.666667,194500,0.005168,NaN


### 한국경제

In [86]:
hk_df = df_total[(df_total.ch_nm == '한국경제TV') & (df_total.st_cd == '005380')]

In [87]:
hk_df = hk_df[hk_df.date.isin(total_table.date)]

In [88]:
hk_text = hk_df.text.values

scores = [] # 긍정 확률
for i in hk_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [89]:
len(scores), len(hk_df)

(18, 18)

In [90]:
hk_df['hk_youtube'] = np.array(scores).reshape(-1,1)
hk_score = hk_df.groupby(['date']).mean().reset_index()[['date','hk_youtube']]

total_table = pd.merge(total_table, hk_score, on='date',how='left')

### 슈카월드

In [91]:
su_df = df_total[(df_total.ch_nm == '슈카월드') & (df_total.st_cd == '005380')]

su_df = su_df[su_df.date.isin(total_table.date)]

su_text = su_df.text.values

scores = [] # 긍정 확률
for i in su_text:
    score, _ = sentiment_predict(i)
    scores.append(score)

In [92]:
len(su_df),len(scores)

(0, 0)

In [93]:
su_df['suka_youtube'] = np.array(scores).reshape(-1,1)
suka_score = su_df.groupby(['date']).mean().reset_index()[['date','suka_youtube']]

total_table = pd.merge(total_table, suka_score, on='date',how='left')

In [94]:
total_table.columns

Index(['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news',
       'mail_news_nsi', 'asia_news', 'asia_news_nsi', 'close', 'change',
       'sampro_youtube', 'hk_youtube', 'suka_youtube'],
      dtype='object')

In [95]:
total_table = total_table[['code', 'date', 'lstm', 'arima', 'fbprophet', 'RL', 'mail_news',
       'mail_news_nsi', 'asia_news', 'asia_news_nsi',
       'sampro_youtube', 'hk_youtube', 'suka_youtube', 'close', 'change']]

# 최종결과

In [96]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,0.643590,128.884615,NaN,0.070443,NaN,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,0.581148,116.324881,0.513158,0.846711,NaN,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,0.413591,82.718254,0.043642,NaN,NaN,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,0.395833,79.416667,0.560342,0.289856,NaN,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,0.534722,106.944444,0.410304,0.271997,NaN,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,0.633333,127.000000,0.439311,88.262285,0.284152,0.444886,NaN,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,0.722222,144.777778,0.828288,0.431312,NaN,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,0.596032,119.206349,0.092859,0.977594,NaN,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,1,0.429420,85.948523,0.412575,82.765031,0.141176,0.092876,NaN,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,0.083333,16.666667,NaN,NaN,NaN,194500,0.005168


In [97]:
while total_table.iloc[-1:].isnull().sum().sum() != 0:
    if total_table.iloc[:1].isnull().sum().sum() != 0:
      for i in range(1, len(total_table)):
        total_table = total_table.fillna(method='pad')
      break
    total_table = total_table.fillna(method='pad')

In [98]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,005380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,0.643590,128.884615,NaN,0.070443,NaN,209000,0.004808
1,005380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,0.581148,116.324881,0.513158,0.846711,NaN,208500,-0.002392
2,005380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,0.413591,82.718254,0.043642,0.846711,NaN,207000,-0.007194
3,005380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,0.395833,79.416667,0.560342,0.289856,NaN,204000,-0.014493
4,005380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,0.534722,106.944444,0.410304,0.271997,NaN,201500,-0.012255
5,005380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,0.633333,127.000000,0.439311,88.262285,0.284152,0.444886,NaN,201500,0.000000
6,005380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,0.722222,144.777778,0.828288,0.431312,NaN,200000,-0.007444
7,005380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,0.596032,119.206349,0.092859,0.977594,NaN,193500,-0.032500
8,005380,2021-10-05,202261.120117,192584.560350,197296.016198,1,0.429420,85.948523,0.412575,82.765031,0.141176,0.092876,NaN,193500,0.000000
9,005380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,0.083333,16.666667,0.141176,0.092876,NaN,194500,0.005168


In [99]:
total_table.to_csv('./total_table.csv', index=False)

In [100]:
total_table = pd.read_csv('./total_table.csv')

In [101]:
total_table_ud = np.round(total_table,2)


for i in range(len(total_table_ud)-1,0,-1):
  if total_table_ud['lstm'].iloc[i] > total_table_ud['lstm'].iloc[i-1]:
    total_table_ud['lstm'].iloc[i] = 1
  else:
    total_table_ud['lstm'].iloc[i] = -1
  
  if total_table_ud['arima'].iloc[i] > total_table_ud['close'].iloc[i-1]:
    total_table_ud['arima'].iloc[i] = 1
  else:
    total_table_ud['arima'].iloc[i] = -1
  
  if total_table_ud['fbprophet'].iloc[i] > total_table_ud['fbprophet'].iloc[i-1]:
    total_table_ud['fbprophet'].iloc[i] = 1
  else:
    total_table_ud['fbprophet'].iloc[i] = -1
  
total_table_ud['mail_news'] = np.where(total_table_ud['mail_news']>0.5,1,
                                       np.where(total_table_ud['mail_news']<0.5,-1,0))

total_table_ud['mail_news_nsi'] = np.where(total_table_ud['mail_news_nsi']>100,1,
                                       np.where(total_table_ud['mail_news_nsi']<100,-1,0))

total_table_ud['asia_news'] = np.where(total_table_ud['asia_news']>0.5,1,
                                       np.where(total_table_ud['asia_news']<0.5,-1,0))

total_table_ud['asia_news_nsi'] = np.where(total_table_ud['asia_news_nsi']>100,1,
                                       np.where(total_table_ud['asia_news_nsi']<100,-1,0))

total_table_ud['sampro_youtube'] = np.where(total_table_ud['sampro_youtube']>0.5,1,
                                       np.where(total_table_ud['sampro_youtube']<0.5,-1,0))

total_table_ud['hk_youtube'] = np.where(total_table_ud['hk_youtube']>0.5,1,
                                       np.where(total_table_ud['hk_youtube']<0.5,-1,0))

total_table_ud['suka_youtube'] = np.where(total_table_ud['suka_youtube']>0.5,1,
                                       np.where(total_table_ud['suka_youtube']<0.5,-1,0))

total_table_ud['change'] = np.where(total_table_ud['change']>0,1,
                                       np.where(total_table_ud['change']<0,-1,0))
total_table_ud.drop('close',1,inplace=True)

In [102]:
total_table_ud

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,change
0,5380,2021-09-17,208241.22,208203.59,207103.53,-1,1,1,1,1,0,-1,0,0
1,5380,2021-09-23,-1.00,1.00,-1.00,-1,1,1,1,1,1,1,0,0
2,5380,2021-09-24,-1.00,-1.00,-1.00,1,-1,-1,-1,-1,-1,1,0,-1
3,5380,2021-09-27,1.00,-1.00,-1.00,-1,-1,-1,-1,-1,1,-1,0,-1
4,5380,2021-09-28,-1.00,1.00,-1.00,1,-1,-1,1,1,-1,-1,0,-1
5,5380,2021-09-29,-1.00,-1.00,-1.00,-1,1,1,-1,-1,-1,-1,0,0
6,5380,2021-09-30,-1.00,1.00,-1.00,1,1,1,1,1,1,-1,0,-1
7,5380,2021-10-01,-1.00,-1.00,-1.00,-1,-1,-1,1,1,-1,1,0,-1
8,5380,2021-10-05,-1.00,-1.00,-1.00,1,-1,-1,-1,-1,-1,-1,0,0
9,5380,2021-10-06,-1.00,1.00,-1.00,1,0,1,-1,-1,-1,-1,0,1


In [103]:
total_table_ud.to_csv('./hun_table_ud.csv', index=False)

# 최종 앙상블 결과

In [104]:
total_table

,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,close,change
0,5380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,0.643590,128.884615,NaN,0.070443,NaN,209000,0.004808
1,5380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,0.581148,116.324881,0.513158,0.846711,NaN,208500,-0.002392
2,5380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,0.413591,82.718254,0.043642,0.846711,NaN,207000,-0.007194
3,5380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,0.395833,79.416667,0.560342,0.289856,NaN,204000,-0.014493
4,5380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,0.534722,106.944444,0.410304,0.271997,NaN,201500,-0.012255
5,5380,2021-09-29,205588.459961,201076.536689,203160.913745,-1,0.633333,127.000000,0.439311,88.262285,0.284152,0.444886,NaN,201500,0.000000
6,5380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,0.722222,144.777778,0.828288,0.431312,NaN,200000,-0.007444
7,5380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,0.596032,119.206349,0.092859,0.977594,NaN,193500,-0.032500
8,5380,2021-10-05,202261.120117,192584.560350,197296.016198,1,0.429420,85.948523,0.412575,82.765031,0.141176,0.092876,NaN,193500,0.000000
9,5380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,0.083333,16.666667,0.141176,0.092876,NaN,194500,0.005168


In [105]:
import statsmodels.api as sm

logit = total_table.copy()
logit = logit.fillna(0)

logit.drop('close',1,inplace=True)
logit['change'] = np.where(logit['change']>0,1,
                           np.where(logit['change']<0,-1,0))

logit = sm.add_constant(logit,has_constant = "add")
a = logit[logit['change'] == 0].index
logit = logit.drop(a)

In [106]:
cols = logit.columns.difference(["change",'code','date'])
X = logit[cols]
y = logit["change"]

In [107]:
# 모델 로딩
import joblib

model = joblib.load('/content/drive/My Drive/[KD1]D1,2_최종 프로젝트_팀별 산출물관리/6조/Code/result/현대차 통합/logit_hun_model.pkl') 
pred_y = model.predict(X)

In [109]:
logit['logit'] = pred_y
ratio = model.predict_proba(X) # [-1,1]
logit['up_ratio'] = 0.1

for i in range(len(logit)):
  logit['up_ratio'].iloc[i] = ratio[i][1] 

In [110]:
logit.to_csv('/content/drive/My Drive/[KD1]D1,2_최종 프로젝트_팀별 산출물관리/6조/Code/result/현대차 통합/hun_logit_table'+end_date+'.csv')

In [111]:
logit

,const,code,date,lstm,arima,fbprophet,RL,mail_news,mail_news_nsi,asia_news,asia_news_nsi,sampro_youtube,hk_youtube,suka_youtube,change,logit,up_ratio
0,1.0,5380,2021-09-17,208241.223633,208203.587355,207103.530647,-1,0.733333,146.966667,0.643590,128.884615,0.000000,0.070443,0.0,1,-1,0.329928
1,1.0,5380,2021-09-23,207684.588867,209323.971210,206236.395237,-1,0.710539,142.401961,0.581148,116.324881,0.513158,0.846711,0.0,-1,-1,0.283164
2,1.0,5380,2021-09-24,207527.737305,208031.123721,205747.992681,1,0.481481,96.296296,0.413591,82.718254,0.043642,0.846711,0.0,-1,-1,0.222104
3,1.0,5380,2021-09-27,207766.279297,206995.430145,204323.300994,-1,0.403645,80.828938,0.395833,79.416667,0.560342,0.289856,0.0,-1,-1,0.188401
4,1.0,5380,2021-09-28,206633.387695,204013.853400,204063.895120,1,0.363095,72.869048,0.534722,106.944444,0.410304,0.271997,0.0,-1,-1,0.245806
6,1.0,5380,2021-09-30,204709.598633,201746.327687,201846.924667,1,0.658163,132.204082,0.722222,144.777778,0.828288,0.431312,0.0,-1,-1,0.334602
7,1.0,5380,2021-10-01,203983.612305,199969.275760,200829.359706,-1,0.090909,18.181818,0.596032,119.206349,0.092859,0.977594,0.0,-1,-1,0.223731
9,1.0,5380,2021-10-06,200252.648438,194011.787988,196040.635361,1,0.500000,101.000000,0.083333,16.666667,0.141176,0.092876,0.0,1,-1,0.201183
